### Uncertainty Quantification in Random Forests vs CatBoost

In [1]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from math import log
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def get_prob_matrix(model, x_test, n_estimators, laplace_smoothing, log=False):
    porb_matrix = [[[] for j in range(n_estimators)] for i in range(x_test.shape[0])]
    for etree in range(n_estimators):
        # populate the porb_matrix with the tree_prob
        tree_prob = model.estimators_[etree].predict_proba(x_test)
        if laplace_smoothing > 0:
            leaf_index_array = model.estimators_[etree].apply(x_test)
            for data_index, leaf_index in enumerate(leaf_index_array):
                leaf_values = model.estimators_[etree].tree_.value[leaf_index]
                leaf_samples = np.array(leaf_values).sum()
                for i,v in enumerate(leaf_values[0]):
                    tmp = (v + laplace_smoothing) / (leaf_samples + (len(leaf_values[0]) * laplace_smoothing))
                    tree_prob[data_index][i] = (v + laplace_smoothing) / (leaf_samples + (len(leaf_values[0]) * laplace_smoothing))

        for data_index, data_prob in enumerate(tree_prob):
            porb_matrix[data_index][etree] = list(data_prob)

        if log:
            print(f"----------------------------------------[{etree}]")
            print(f"class {model.estimators_[etree].predict(x_test)}  prob \n{tree_prob}")
    return porb_matrix

def uncertainty_estimate(probs): # three dimentianl array with d1 as datapoints, (d2) the rows as samples and (d3) the columns as probability for each class
    p = np.array(probs)
    entropy = -p*np.ma.log10(p)
    entropy = entropy.filled(0)
    a = np.sum(entropy, axis=1)
    a = np.sum(a, axis=1) / entropy.shape[1]
    p_m = np.mean(p, axis=1)
    total = -np.sum(p_m*np.ma.log10(p_m), axis=1)
    total = total.filled(0)
    e = total - a
    return total, e, a

def transpose_stage_predictions(staged_preds):
    
    model_probas = list(staged_preds)
    mbier
    for i in range(len(model_probas[0])):
        model_probas_transposed.append([el[i] for el in model_probas])
    return model_probas_transposed

#### Load SPECT Dataset

In [3]:
train = pd.read_csv('data/SPECT .train', delimiter=',', header=None)
test = pd.read_csv('data/SPECT .test', delimiter=',',header=None)
data = pd.concat([train,test]).reset_index(drop=True)
x = data.drop(0, axis=1)
y = data[0]

In [29]:
rf_ARC_eU_list, rf_ARC_aU_list, rf_ARC_random_list = [],[],[]
cb_ARC_eU_list, cb_ARC_aU_list, cb_ARC_random_list = [],[],[]

rf_criterion='entropy'
rf_max_depth=10
rf_n_estimators=50
rf_laplace_smoothing = 1

cb_n_trees = 25
cb_max_depth = 3
test_size=0.3


for i in range(100):
    
    # random train_test split
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size,shuffle=True)
    x_train, x_test, y_train, y_test = x_train.copy(), x_test.copy(), y_train.copy(), y_test.copy()

    
    # init models
    rf_model = RandomForestClassifier(bootstrap=True,
                                        criterion=rf_criterion,
                                        max_depth=rf_max_depth,
                                        n_estimators=rf_n_estimators,
                                        random_state=None,
                                        verbose=0,
                                        warm_start=False)
    
    cb_model = CatBoostClassifier(iterations=cb_n_trees,
                               learning_rate=0.5,
                               depth=cb_max_depth,
                              cat_features=x_train.columns-1,
                              silent=True)
    
    
    # fit models
    rf_model.fit(x_train, y_train)
    cb_model.fit(x_train, y_train)

    # let models predict test data
    rf_preds_class = rf_model.predict(x_test)
    cb_preds_class = cb_model.predict(x_test)

    # evaluate accuracy
    rf_accuracy = accuracy_score(y_test, rf_preds_class)
    cb_accuracy = accuracy_score(y_test, cb_preds_class)
    
    # calculate uncertainties
    rf_prob_matrix = get_prob_matrix(rf_model, x_test, rf_n_estimators, rf_laplace_smoothing, log=0)
    rf_total_uncertainty, rf_epistemic_uncertainty, rf_aleatoric_uncertainty = uncertainty_estimate(np.array(rf_prob_matrix))
    
    cb_staged_preds = cb_model.staged_predict(x_test,
                   prediction_type='Probability',
                   ntree_start=0, 
                   ntree_end=10)
    cb_prob_matrix = transpose_stage_predictions(cb_staged_preds)
    cb_total_uncertainty, cb_epistemic_uncertainty, cb_aleatoric_uncertainty = uncertainty_estimate(np.array(cb_prob_matrix))
    
    # append model uncertainties to samples
    rf_eU = x_test.copy()
    rf_aU = x_test.copy()
    cb_eU = x_test.copy()
    cb_aU = x_test.copy()
    
    rf_eU ['e'] = rf_epistemic_uncertainty
    rf_aU ['a'] = rf_aleatoric_uncertainty
    
    cb_eU ['e'] = cb_epistemic_uncertainty
    cb_aU ['a'] = cb_aleatoric_uncertainty
                    
    
    # sort samples by specific uncertainties
    rf_aU.sort_values('a', ascending=True, inplace=True)
    rf_eU.sort_values('e', ascending=True, inplace=True)
    
    cb_aU.sort_values('a', ascending=True, inplace=True)
    cb_eU.sort_values('e', ascending=False, inplace=True)
    
    # init lists, which will contain accuracy of each classifier
    # for each rejection rate after loop
    
    rf_ARC_eU = [rf_accuracy]
    rf_ARC_aU = [rf_accuracy]
    rf_ARC_random = [rf_accuracy]
    cb_ARC_eU = [cb_accuracy]
    cb_ARC_aU = [cb_accuracy]
    cb_ARC_random = [cb_accuracy]
    
    # init params for ARC
    num_rows = x_test.shape[0]
    discard_count = 0
    step_size = 0.05
    
    
    current_step = 0.05
    while current_step < 1:
        discard_count = int (num_rows*current_step)
        
        # select a portion i of samples, for which uc is lowest 
        rf_eU_samples = rf_eU.iloc[:-discard_count,0:22]
        rf_aU_samples = rf_aU.iloc[:-discard_count,0:22]
        
        cb_eU_samples = cb_eU.iloc[:-discard_count,0:22]
        cb_aU_samples = cb_aU.iloc[:-discard_count,0:22]
        
        # select a portion i of samples randomly
        random_samples = x_test.iloc[:,0:22].sample((num_rows-discard_count))
        
        # let models predict selected samples
        rf_preds_class_eU = rf_model.predict(rf_eU_samples)
        rf_preds_class_aU = rf_model.predict(rf_aU_samples)
        rf_preds_class_random = rf_model.predict(random_samples)
        
        cb_preds_class_eU = cb_model.predict(cb_eU_samples)
        cb_preds_class_aU = cb_model.predict(cb_aU_samples)
        cb_preds_class_random = cb_model.predict(random_samples)
        
        # evaluate models regarding accuracy
        rf_accuracy_eu = accuracy_score(y_test[rf_eU_samples.index], rf_preds_class_eU)
        rf_accuracy_au = accuracy_score(y_test[rf_aU_samples.index], rf_preds_class_aU)
        rf_accuracy_random = accuracy_score(y_test[random_samples.index], rf_preds_class_random)
        
        cb_accuracy_eu = accuracy_score(y_test[cb_eU_samples.index], cb_preds_class_eU)
        cb_accuracy_au = accuracy_score(y_test[cb_aU_samples.index], cb_preds_class_aU)
        cb_accuracy_random = accuracy_score(y_test[random_samples.index], cb_preds_class_random)
        
        # append eval results to list 
        rf_ARC_eU.append(rf_accuracy_eu)
        rf_ARC_aU.append(rf_accuracy_au)
        rf_ARC_random.append(rf_accuracy_random)
        
        cb_ARC_eU.append(cb_accuracy_eu)
        cb_ARC_aU.append(cb_accuracy_au)
        cb_ARC_random.append(cb_accuracy_random)
        
        current_step+=step_size
    
    # append list of eval results for one iteration to list
    rf_ARC_eU_list.append(rf_ARC_eU)
    rf_ARC_aU_list.append(rf_ARC_aU)
    rf_ARC_random_list.append(rf_ARC_random)
    
    cb_ARC_eU_list.append(cb_ARC_eU)
    cb_ARC_aU_list.append(cb_ARC_aU)
    cb_ARC_random_list.append(cb_ARC_random)

In [30]:
# calculate mean values over all iterations for  both classifiers

rf_ARC_eU_mean, rf_ARC_aU_mean, rf_ARC_random_mean = [],[],[]
cb_ARC_eU_mean, cb_ARC_aU_mean, cb_ARC_random_mean = [],[],[]

for i in range(len(rf_ARC_eU_list[0])):
    rf_ARC_eU_mean.append(np.array([el[i] for el in rf_ARC_eU_list]).mean())
    rf_ARC_aU_mean.append(np.array([el[i] for el in rf_ARC_aU_list]).mean())
    rf_ARC_random_mean.append(np.array([el[i] for el in rf_ARC_random_list]).mean())
    
    cb_ARC_eU_mean.append(np.array([el[i] for el in cb_ARC_eU_list]).mean())
    cb_ARC_aU_mean.append(np.array([el[i] for el in cb_ARC_aU_list]).mean())
    cb_ARC_random_mean.append(np.array([el[i] for el in cb_ARC_random_list]).mean())

#### Tabular Results

In [31]:
print('Random Forest')
print('Reject   E    A    R ')
i = 0.00
for e,a,r in zip(rf_ARC_eU_mean,rf_ARC_aU_mean,rf_ARC_random_mean):
    print(f'{i:.2f}:  {e:.2f} {a:.2f} {r:.2f} ')
    i +=step_size

print()
print('CatBoost')
print('Reject   E    A    R ')
i = 0.00
for e,a,r in zip(cb_ARC_eU_mean,cb_ARC_aU_mean,cb_ARC_random_mean):
    print(f'{i:.2f}:  {e:.2f} {a:.2f} {r:.2f} ')
    i +=step_size

Random Forest
Reject   E    A    R 
0.00:  0.82 0.82 0.82 
0.05:  0.82 0.83 0.82 
0.10:  0.83 0.84 0.82 
0.15:  0.83 0.85 0.82 
0.20:  0.84 0.88 0.82 
0.25:  0.84 0.90 0.82 
0.30:  0.85 0.92 0.82 
0.35:  0.85 0.93 0.82 
0.40:  0.86 0.94 0.83 
0.45:  0.86 0.94 0.82 
0.50:  0.87 0.94 0.81 
0.55:  0.87 0.95 0.83 
0.60:  0.88 0.96 0.82 
0.65:  0.89 0.96 0.82 
0.70:  0.90 0.97 0.82 
0.75:  0.90 0.97 0.82 
0.80:  0.91 0.99 0.81 
0.85:  0.91 0.99 0.82 
0.90:  0.94 1.00 0.83 
0.95:  0.99 1.00 0.81 

CatBoost
Reject   E    A    R 
0.00:  0.82 0.82 0.82 
0.05:  0.83 0.83 0.82 
0.10:  0.83 0.84 0.82 
0.15:  0.84 0.85 0.82 
0.20:  0.85 0.86 0.82 
0.25:  0.86 0.88 0.82 
0.30:  0.86 0.89 0.82 
0.35:  0.86 0.91 0.82 
0.40:  0.87 0.92 0.82 
0.45:  0.87 0.93 0.82 
0.50:  0.88 0.94 0.81 
0.55:  0.88 0.95 0.82 
0.60:  0.89 0.96 0.82 
0.65:  0.89 0.96 0.82 
0.70:  0.89 0.97 0.82 
0.75:  0.89 0.98 0.81 
0.80:  0.88 0.99 0.80 
0.85:  0.88 0.99 0.82 
0.90:  0.87 0.99 0.83 
0.95:  0.84 1.00 0.81 


#### Plot Results

In [32]:
steps = np.arange(0.0, 1.0, step_size)
directory = 'results/'
plot_name = 'CatBoost - Epistemische Unsicherheit Inverted'

plt.plot(steps , cb_ARC_eU_mean, label="CatBoost") #, marker='o', linestyle='--' , color='r'
plt.plot(steps , cb_ARC_random_mean, label="Random")
plt.xlabel('Rejection %')
plt.ylabel('Accuracy %') 
plt.title('')  #'Accuracy-Rejection curve'
plt.legend() 
plt.savefig(f"{directory}{plot_name}.png")
plt.close()

In [125]:
steps = np.arange(0.0, 1.0, step_size)
directory = 'results/'
plot_name = 'Random Forest - Epistemische Unsicherheit'

plt.plot(steps , rf_ARC_eU_mean, label="Random Forest") #, marker='o', linestyle='--' , color='r'
plt.plot(steps , rf_ARC_random_mean, label="Random")
plt.xlabel('Rejection %')
plt.ylabel('Accuracy %') 
plt.title('') #'Accuracy-Rejection curve'
plt.legend() 
plt.savefig(f"{directory}{plot_name}.png")
plt.close()

In [126]:
steps = np.arange(0.0, 1.0, step_size)
directory = 'results/'
plot_name = 'CatBoost - Epistemische Unsicherheit'

plt.plot(steps , cb_ARC_eU_mean, label="CatBoost") #, marker='o', linestyle='--' , color='r'
plt.plot(steps , cb_ARC_random_mean, label="Random")
plt.xlabel('Rejection %')
plt.ylabel('Accuracy %') 
plt.title('')  #'Accuracy-Rejection curve'
plt.legend() 
plt.savefig(f"{directory}{plot_name}.png")
plt.close()

In [127]:
steps = np.arange(0.0, 1.0, step_size)
directory = 'results/'
plot_name = 'Random Forest - Aleatorische Unsicherheit'

plt.plot(steps , rf_ARC_aU_mean, label="Random Forest") #, marker='o', linestyle='--' , color='r'
plt.plot(steps , rf_ARC_random_mean, label="Random")
plt.xlabel('Rejection %')
plt.ylabel('Accuracy %') 
plt.title('')   #'Accuracy-Rejection curve'
plt.legend() 
plt.savefig(f"{directory}{plot_name}.png")
plt.close()

In [128]:
steps = np.arange(0.0, 1.0, step_size)
directory = 'results/'
plot_name = 'CatBoost - Aleatorische Unsicherheit'

plt.plot(steps , cb_ARC_aU_mean, label="CatBoost") #, marker='o', linestyle='--' , color='r'
plt.plot(steps , cb_ARC_random_mean, label="Random")
plt.xlabel('Rejection %')
plt.ylabel('Accuracy %') 
plt.yticks(np.arange(0.8, 1, 0.02))
plt.title('')   #'Accuracy-Rejection curve'
plt.legend() 
plt.savefig(f"{directory}{plot_name}.png")
plt.close()

In [129]:
steps = np.arange(0.0, 1.0, step_size)
directory = 'results/'
plot_name = 'Aleatorische Unsicherheit - CatBoost vs Random Forest'

plt.plot(steps , rf_ARC_aU_mean, label="Random Forest") #, marker='o', linestyle='--' , color='r'
plt.plot(steps , cb_ARC_aU_mean, label="CatBoost")
plt.xlabel('Rejection %')
plt.ylabel('Accuracy %') 
plt.title('')   #'Accuracy-Rejection curve'
plt.legend() 
plt.savefig(f"{directory}{plot_name}.png")
plt.close()

In [130]:
steps = np.arange(0.0, 1.0, step_size)
directory = 'results/'
plot_name = 'Epistemische Unsicherheit - CatBoost vs Random Forest'

plt.plot(steps , rf_ARC_eU_mean, label="Random Forest") #, marker='o', linestyle='--' , color='r'
plt.plot(steps , cb_ARC_eU_mean, label="CatBoost")
plt.xlabel('Rejection %')
plt.ylabel('Accuracy %') 
plt.title('')   #'Accuracy-Rejection curve'
plt.legend() 
plt.savefig(f"{directory}{plot_name}.png")
plt.close()